In [1]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize as wtoken
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize


import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import Input
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import (Activation, Dropout, Dense, Embedding, 
                                     Flatten, BatchNormalization, GlobalMaxPooling1D, 
                                     GRU, Lambda, concatenate, LSTM, Concatenate, Conv1D,
                                    Reshape, BatchNormalization)
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from tensorflow.keras import regularizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import os, string, gc, re

from tqdm import tqdm


import tldextract as tld


from wordcloud import WordCloud
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# A quick verification to see if the gpu drivers are ready to serve traning DL
print("Tensorflow Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Tensorflow Version:  2.0.0
Eager mode:  True
Hub version:  0.7.0
GPU is available
Num GPUs Available:  1


In [3]:
path = '/media/moh/abunayla/Disaster_Tweets/Notebooks/data/'

In [4]:
train = pd.read_csv(path + 'train.csv', usecols = ['text'])

test = pd.read_csv(path + 'test.csv', usecols = ['text'])

samsub = pd.read_csv(path +'sample_submission.csv', index_col = 'id')

Y =  pd.read_csv(path +'train.csv', usecols = ['target'])

In [5]:
train.head()

,text
0,Our Deeds are the Reason of this #earthquake M...
1,Forest fire near La Ronge Sask. Canada
2,All residents asked to 'shelter in place' are ...
3,"13,000 people receive #wildfires evacuation or..."
4,Just got sent this photo from Ruby #Alaska as ...


In [6]:
train.shape

(7613, 1)

In [7]:
maxlen =300
training_samples =6000
max_words = 15000

In [8]:
#x_train = train[:training_samples].values
#x_train = x_train.reshape((x_train.shape[0],))
x_train = train.values.reshape(train.shape[0],)
x_train.shape

(7613,)

In [9]:
#y_train = Y[:training_samples].values
#y_train = y_train.reshape((y_train.shape[0],))

y_train = Y.values.reshape((Y.shape[0],))
y_train.shape

(7613,)

In [10]:
#x_val = train[training_samples:].values
#x_val = x_val.reshape((x_val.shape[0],))
#x_val.shape

In [11]:
#y_val = Y[training_samples:].values
#y_val = y_val.reshape((y_val.shape[0],))
#y_val.shape

In [12]:
#len(y_val) == len(x_val)

In [13]:
#model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
nnlm = 'https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1'
use4 = path + 'USE4'

#hub_layer(x_train[:3])

In [14]:
hub_layer = hub.KerasLayer(use4, output_shape=[512], input_shape=[], 
                           dtype=tf.string, trainable=True)

In [15]:
model = Sequential(
    [
        hub_layer,
        Reshape((512, 1), input_shape = (512, )),
        GRU(25, activation = 'tanh', name='h1', return_sequences = True),
        Dropout(0.3),
        GRU(100, activation = 'tanh', name = 'h2'),
        Dropout(0.3),
        Dense(1, activation = 'sigmoid')
    ]
)

model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['acc'])

In [16]:
train_batches = np.array_split(x_train, 10)
targets_batches = np.array_split(y_train, 10)

In [17]:
for train_batch,target_batch in zip(train_batches, targets_batches):
    assert train_batch.shape == target_batch.shape


In [18]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [19]:
history = model.fit(x_train,
                    y_train,
                    epochs=10,
                    batch_size=256,
                    validation_split=0.6,
                    shuffle = True,
                    verbose=1)

Train on 3045 samples, validate on 4568 samples
Epoch 1/10
 256/3045 [=>............................] - ETA: 1:31

NotFoundError:  No registered 'HashTableV2' OpKernel for 'GPU' devices compatible with node {{node text_preprocessor/hash_table}}
	.  Registered:  device='CPU'; key_dtype in [DT_STRING]; value_dtype in [DT_STRING]
  device='CPU'; key_dtype in [DT_STRING]; value_dtype in [DT_INT64]
  device='CPU'; key_dtype in [DT_STRING]; value_dtype in [DT_INT32]
  device='CPU'; key_dtype in [DT_STRING]; value_dtype in [DT_FLOAT]
  device='CPU'; key_dtype in [DT_STRING]; value_dtype in [DT_DOUBLE]
  device='CPU'; key_dtype in [DT_STRING]; value_dtype in [DT_BOOL]
  device='CPU'; key_dtype in [DT_INT64]; value_dtype in [DT_STRING]
  device='CPU'; key_dtype in [DT_INT64]; value_dtype in [DT_INT64]
  device='CPU'; key_dtype in [DT_INT64]; value_dtype in [DT_INT32]
  device='CPU'; key_dtype in [DT_INT64]; value_dtype in [DT_FLOAT]
  device='CPU'; key_dtype in [DT_INT64]; value_dtype in [DT_DOUBLE]
  device='CPU'; key_dtype in [DT_INT32]; value_dtype in [DT_STRING]
  device='CPU'; key_dtype in [DT_INT32]; 

In [ ]:
model.save_weights(path+'dense_gnews-swivel-20dim.h5')

In [ ]:
acc = history.history['acc']

val_acc = history.history['val_acc']

loss = history.history['loss']

val_loss = history.history['val_loss']

epochs= range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training Accuracy')



In [ ]:
plt.plot(epochs, val_acc, 'b', label = 'Validation')

In [ ]:
preds = model.predict(test_data)
preds = np.around(preds)
preds = preds.astype(int)

In [ ]:
samsub.target = preds

In [ ]:
samsub.head()

In [ ]:
samsub.to_csv('submission.csv')

# Your Best Entry 

You advanced 584 places on the leaderboard!

Your submission scored 0.80879, which is an improvement of your previous score of 0.79447. Great job!